# Part 8 bis - Introduction to Protocols

### Context 

Now that we've been through Plans, we'll introduce a new object called the Protocol. A Protocol coordinates a sequence of Plans, deploys them on distant workers
and run them in a single pass.

It's a high level object which contains the logic of a complex computation
distributed across several workers. The main feature of Protocol is the
ability to be sent / searched / fetched back between workers, and finally
deployed to identified workers. So a user can design a protocol, upload it
to a cloud worker, and any other workers will be able to search for it,
download it, and apply the computation program it contains on the workers
that it is connected to.

Let's see how to use it!

Authors:
- Théo Ryffel - Twitter [@theoryffel](https://twitter.com/theoryffel) - GitHub: [@LaRiffle](https://github.com/LaRiffle)

### 1. Create and deploy

Protocol are created by providing a list of pairs `(worker, plan)`. `worker` can be either a real
worker or a worker id or a string to represent a fictive worker. This
last case can be used at creation to specify that two plans should be
owned (or not owned) by the same worker at deployment. `plan` can
either be a Plan or a PointerPlan.

In [1]:
import torch as th
import syft as sy
hook = sy.TorchHook(th)

# IMPORTANT: Local worker should not be a client worker
hook.local_worker.is_client_worker = False

/home/billy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/billy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/billy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/billy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

/home/billy/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/billy/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/billy/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/billy/.local/lib/python3.6/site-packages/tensorboard/compat/tensorf

Let's define 3 plans and feed them to a protocol. They all perform an increment operation.

In [2]:
@sy.func2plan(args_shape=[(1,)])
def inc1(x):
    return x + 1

@sy.func2plan(args_shape=[(1,)])
def inc2(x):
    return x + 1

@sy.func2plan(args_shape=[(1,)])
def inc3(x):
    return x + 1

protocol = sy.Protocol([("worker1", inc1), ("worker2", inc2), ("worker3", inc3)])

Now we need to bind the Protocol to workers, which is done by calling `.deploy(*workers)`. Let's create some workers.

In [3]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
charlie = sy.VirtualWorker(hook, id="charlie")

In [4]:
workers = alice, bob, charlie

protocol.deploy(*workers)

<Protocol id:18622180517 owner:me resolved>
 - alice: [PointerPlan | me:79255873099 -> alice:91490922461]
 - bob: [PointerPlan | me:91424454542 -> bob:70424938391]
 - charlie: [PointerPlan | me:90951282817 -> charlie:94846935866]

You can see that the plans have already been sent to the appropriate workers: it has been deployed!

This has been done in 2 phases: first, we map the fictive workers provided at creation
(named by strings) to the provided workers, and second, we send the corresponding
plans to each of them.

### 2. Run a protocol

Running a protocol means executing all the plans sequentially. To do so, you provide some input data which is sent to the first plan location. This first plan is
run and its output is moved to the second plan location, and so on. The final
result is returned after all plans have run, and it is composed of pointers to
the last plan location.

In [5]:
x = th.tensor([1.0])
ptr = protocol.run(x)
ptr

send alice
move alice  ->  bob
move bob  ->  charlie


(Wrapper)>[PointerTensor | me:81097315012 -> charlie:88799914664]

In [6]:
ptr.get()

(Wrapper)>(Wrapper)>tensor([4.])

The input 1.0 has been through the 3 plans and so has been incremented 3 times, that's why it now equals 4.0!

Actually, you can also **run a protocol remotely** on some pointers to data:

In [ ]:
james = sy.VirtualWorker(hook, id="james")

In [ ]:
protocol.send(james)

In [ ]:
x = th.tensor([1.0]).send(james)
ptr = protocol.run(x)
ptr

As you see the result is a pointer to james

In [ ]:
ptr = ptr.get()
ptr

In [ ]:
ptr = ptr.get()
ptr

### 3. Search for a protocol

In real settings you might want to download a remote protocol, to deploy it on your workers and to run it with you data:

Let's initialize a protocol **which is not deployed**, and put it on a remote worker

In [ ]:
protocol = sy.Protocol([("worker1", inc1), ("worker2", inc2), ("worker3", inc3)])
protocol.tag('my_protocol')
protocol.send(james)

In [ ]:
me = sy.hook.local_worker # get access to me as a local worker

Now we launch a search to find the protocol

In [ ]:
responses = me.request_search(['my_protocol'], location=james)
responses

You have access to a pointer to a Protocol

In [ ]:
ptr_protocol = responses[0]

Like usual pointer you can get it back:

In [ ]:
protocol_back = ptr_protocol.get()
protocol_back

And we can do like we did in parts 1. & 2.

In [ ]:
protocol_back.deploy(alice, bob, charlie)

x = th.tensor([1.0])
ptr = protocol_back.run(x)
ptr.get()

More real world examples will come with Protocols, but you can already see all the possibilities opened by this new object!

### Star PySyft on GitHub

The easiest way to help our community is just by starring the repositories! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Pick our tutorials on GitHub!

We made really nice tutorials to get a better understanding of what Federated and Privacy-Preserving Learning should look like and how we are building the bricks for this to happen.

- [Checkout the PySyft tutorials](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! 

- [Join slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! If you want to start "one off" mini-projects, you can go to PySyft GitHub Issues page and search for issues marked `Good First Issue`.

- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

- [Donate through OpenMined's Open Collective Page](https://opencollective.com/openmined)